In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/Tomato Plant'

/content/gdrive/MyDrive/Tomato Plant


In [ ]:
import os
import cv2
import numpy as np

data_dir = "tomato/train/"
classes = ["healthy", "diseased"]
img_size = (224, 224)

data = []

for cls in classes:
    folder_path = os.path.join(data_dir, cls)
    label = classes.index(cls)
    
    for img_path in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        data.append([img, label])
        
np.random.shuffle(data)

X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)
    
X = np.array(X)
y = np.array(y)


In [ ]:
# The preprocessing steps were already done during data collection.
# Ensure that the images are in the format (batch_size, height, width, channels)
X = np.reshape(X, (X.shape[0], img_size[0], img_size[1], 3))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        self.target_update_freq = 1000
        self.steps = 0
        
        self.model = self.build_model()
        self.target_model = self.build_model()
        
    def build_model(self):
        inputs = layers.Input(shape=self.state_size)
        x = layers.Conv2D(32, (3, 3), activation="relu")(inputs)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation="relu")(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Flatten()(x)
        x = layers.Dense(64, activation="relu")(x)
        outputs = layers.Dense(self.action_size, activation="softmax")(x)
        model = models.Model(inputs=inputs, outputs=outputs)
        model.compile(loss="mse", optimizer=optimizers.Adam(lr=0.001))
        return model
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            return np.argmax(self.model.predict(state))
        
    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = np.random.choice(np.arange(len(self.memory)), size=self.batch_size, replace=False)
        minibatch = [self.memory[i] for i in minibatch]

        for state, action, reward, next_state, done in minibatch:
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(next_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)


In [ ]:
import tensorflow as tf

# Force TensorFlow to use the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [ ]:
state_size = (img_size[0], img_size[1], 3)
action_size = len(classes)

agent = DQNAgent(state_size, action_size)

num_episodes = 100
steps_per_episode = X.shape[0] // num_episodes

for episode in range(num_episodes):
    for step in range(steps_per_episode):
        idx = episode * steps_per_episode + step
        state = X[idx:idx+1]
        action = agent.act(state)
        next_state = state
        reward = y[idx]
        done = True if step == steps_per_episode - 1 else False
        agent.remember(state, action, reward, next_state, done)
        agent.replay()

    print("Episode: {}/{}, Steps: {}".format(episode + 1, num_episodes, steps_per_episode))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step
Episode: 49/100, Steps: 2
1/1 [==============================] - 0s 19ms/step
Episode: 50/100, Steps: 2
1/1 [==============================] - 0s 18ms/step
Episode: 51/100, Steps: 2
1/1 [==============================] - 0s 17ms/step
Episode: 52/100, Steps: 2
1/1 [==============================] - 0s 18ms/step
Episode: 53/100, Steps: 2
1/1 [==============================] - 0s 19ms/step
Episode: 54/100, Steps: 2
1/1 [==============================] - 0s 19ms/step
Episode: 55/100, Steps: 2
1/1 [==============================] - 0s 20ms/step
Episode: 56/100, Steps: 2
1/1 [==============================] - 0s 20ms/step
Episode: 57/100, Steps: 2
1/1 [==============================] - 0s 20ms/step
Episode: 58/100, Steps: 2
1/1 [==============================] - 0s 19ms/step
Episode: 59/100, Steps: 2
1/1 [==============================] - 0s 17ms/step
Episode: 60/100, Steps: 2
1/1 [========

In [ ]:
test_data_dir = "tomato/val/"

test_data = []

for cls in classes:
    folder_path = os.path.join(test_data_dir, cls)
    label = classes.index(cls)
    
    for img_path in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        test_data.append([img, label])
        
np.random.shuffle(test_data)

X_test = []
y_test = []

for feature, label in test_data:
    X_test.append(feature)
    y_test.append(label)
    
X_test = np.array(X_test)
y_test = np.array(y_test)

X_test = np.reshape(X_test, (X_test.shape[0], img_size[0], img_size[1], 3))

correct = 0

for i in range(X_test.shape[0]):
    state = X_test[i:i+1]
    action = agent.act(state)
    if action == y_test[i]:
        correct += 1
        
accuracy = correct / X_test.shape[0]

print("Accuracy:", accuracy)


NameError: ignored

In [ ]:
# Load a single test image
img = cv2.imread("tomato/val/healthy/01c1da17-8d9f-4d69-8a1e-58d37453d3c3___RS_HL 9641.JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, img_size)
img = img / 255.0

# Reshape the image to match the input shape of the DQN agent
state = np.reshape(img, (1, img_size[0], img_size[1], 3))

# Use the act() method of the DQN agent to predict the class of the image
action = agent.act(state)

# Map the predicted action to the corresponding class label
predicted_class = classes[action]

print("Predicted class:", predicted_class)

# Load a single test image
img = cv2.imread("tomato/val/diseased/bacterial_spot (26).JPG")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, img_size)
img = img / 255.0

# Reshape the image to match the input shape of the DQN agent
state = np.reshape(img, (1, img_size[0], img_size[1], 3))

# Use the act() method of the DQN agent to predict the class of the image
action = agent.act(state)

# Map the predicted action to the corresponding class label
predicted_class = classes[action]

print("Predicted class:", predicted_class)


Predicted class: healthy
Predicted class: diseased
